# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770833535136                   -0.52    9.0         
  2   -2.772141716361       -2.88       -1.32    1.0    153ms
  3   -2.772170160315       -4.55       -2.52    1.0    156ms
  4   -2.772170688813       -6.28       -3.35    2.0    199ms
  5   -2.772170720058       -7.51       -3.80    1.0    160ms
  6   -2.772170722695       -8.58       -4.53    1.0    183ms
  7   -2.772170723000       -9.52       -4.87    2.0    185ms
  8   -2.772170723014      -10.85       -5.65    1.0    174ms
  9   -2.772170723015      -12.12       -6.72    2.0    202ms
 10   -2.772170723015      -14.24       -6.99    1.0    188ms
 11   -2.772170723015      -14.18       -7.91    1.0    174ms
 12   -2.772170723015   +  -14.75       -8.16    2.0    211ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580155357455

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770922004076                   -0.53    9.0         
  2   -2.772064635035       -2.94       -1.32    1.0    198ms
  3   -2.772083204154       -4.73       -2.51    1.0    154ms
  4   -2.772083365351       -6.79       -3.25    1.0    183ms
  5   -2.772083415731       -7.30       -3.86    2.0    217ms
  6   -2.772083417730       -8.70       -4.78    1.0    162ms
  7   -2.772083417808      -10.10       -5.28    2.0    214ms
  8   -2.772083417810      -11.69       -6.06    1.0    168ms
  9   -2.772083417811      -12.69       -6.93    2.0    208ms
 10   -2.772083417811      -14.75       -7.62    1.0    178ms
 11   -2.772083417811   +  -15.35       -8.23    2.0    200ms

Polarizability via ForwardDiff:       1.7725349682992868
Polarizability via finite difference: 1.7735580155357455
